# Classifying Identifiable People Using Image Classification
Identify whether individuals in an image are "identifiable" or "unidentifiable". Images are considered "identifiable" if they clearly show an individual's face with no obstructions (mask, sunglasses, etc.). An applicaton of this project may be for companies that need to keep track of everyone visiting their locations.

## Step 1: Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

## Step 2: Load the Data

In [ ]:
labels = ['identifiable', 'unidentifiable']
img_size = 224

def get_data(data_dir):
    """Return a numpy array

    Numpy array containing train or testing set data values. Read the images from 
    data_dir in the RGB format and resize the images to the desired width and height.
    """
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label) # 0: identifiable, 1: unidentifiable
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data, dtype=object)

In [ ]:
# Fetch the train and validation data.
train = get_data('images dataset/Training Set')
val = get_data('images dataset/Testing Set')

## Step 3: Visualize the data

In [ ]:
l = []
for i in train:
    if(i[1] == 0):
        l.append("identifiable")
    else:
        l.append("unidentifiable")
sns.set_style('darkgrid')
sns.countplot(x=l)

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(train[1][0])
plt.title(labels[train[0][1]])

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(train[-1][0])
plt.title(labels[train[-1][1]])

## Step 4: Data Preprocessing and Augmentation

In [ ]:
# Preprocess the data
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [ ]:
# Augment the data
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

## Step 5: Define the Model

In [ ]:
# Define a simple CNN with 3 Convolutional layers followed by max-pooling layers. 
#   A dropout layer is added after the 3rd maxpool operation to avoid overfitting.
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(224,224,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
# Compile the model using the Adam optimizer.
#   SparseCategoricalCrossentropy is the loss function.
#   Use a lower learning rate of 0.000001 for a smoother curve.
opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [ ]:
# epochs represent the number of times the model goes through the training set.
#   The model can be updated multiple times during one epoch.
#   Train the model for 500 epochs because the learning rate is so small.
#      500 epochs will produce a runtime of nearly 4 hours on a local machine.
history = model.fit(x_train,y_train,epochs = 5 , validation_data = (x_val, y_val))